## define functions

- contains requered number -> ensure only specific images are pull
- get image -> pull the require and compress image
- scrapp_image_description -> iterate for each of the products
- grucci -> iterate scrap_image

### scrapper

In [1]:
import requests
from PIL import Image
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
from io import BytesIO
import pandas as pd
from openpyxl import load_workbook

from selenium.webdriver import Chrome
from selenium.webdriver.edge.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time


from azure.storage.blob import BlobServiceClient, generate_blob_sas, BlobSasPermissions
from azure_key import account_name, account_key, container_name

import psycopg2
from sql_key import dbname, user, password, host, port


In [46]:
def create_soup(url):

    chrome_options = Options()
    #chrome_options.add_argument("--headless")  # Ensure GUI is off
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")


    browser = Chrome(options=chrome_options)

    try:
        # Open the URL
        browser.maximize_window()
        browser.get(url)
        
        # Wait for the page to fully load
        wait = WebDriverWait(browser, 120)
        wait.until(lambda driver: driver.execute_script('return document.readyState') == 'complete')
        
        # Wait for the cookie button to be clickable
        cookie_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button.Button_button__qoiWI.CookiesContent_button__2Vj_R')))
        cookie_button.click()

        # Scroll to the bottom of the page
        last_height = browser.execute_script("return document.body.scrollHeight")
        
        while True:
            for i in range(5):  # 5 iterations to make 20% scroll each
                browser.execute_script("window.scrollBy(0, document.body.scrollHeight / 5);")
                time.sleep(1)  # Wait for 1 second between each scroll
            
            # Calculate new scroll height and compare with the last scroll height
            new_height = browser.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height


        # Get the page source after scrolling
        html = browser.page_source

    except Exception as error:
        print(f"Error: {error}")
 
    browser.quit()
     


    page_soup = BeautifulSoup(html, 'html.parser')

    return page_soup


In [5]:
def connect_blob():

    connect_str = 'DefaultEndpointsProtocol=https;AccountName='+account_name+';AccountKey='+account_key+';EndpointSuffix=core.windows.net'

    #use the client to connect
    blob_Service_client = BlobServiceClient.from_connection_string(connect_str)

    #Use the client to connect to the container 
    container_client = blob_Service_client.get_container_client(container_name)

    return container_client

In [6]:
def connect_sql():
    # Database connection parameters
    conn_params = {
        'dbname': dbname[0],
        'user': user[0],
        'password': password[0],
        'host': host[0],
        'port': port
    }
    # Connect to the PostgreSQL server
    conn = psycopg2.connect(**conn_params)
    cursor = conn.cursor()

    return conn, cursor

In [7]:
def images_to_blob(image, image_name,container_client):

    try:
        img_byte_arr = BytesIO()
        image.save(img_byte_arr, format=image.format)
        img_byte_arr = img_byte_arr.getvalue()

        container_client.upload_blob(name= image_name,data=img_byte_arr )
    except Exception as error:
        print(f"Error: {error}")
    

In [8]:
def data_to_SQL(prod_name, prod_description,brand,prod_images_names,cursor,conn):

    try:
        # Insert in Brand_Prod_table
        products_data = [('BrandA', 'Description for BrandA product')]

        insert_products_query = """
        INSERT INTO Products (Brand, Descript) VALUES (%s, %s) RETURNING Brand_Prod_id;
        """
        cursor.execute(insert_products_query)

        # Insert in Image mapping
        insert_characteristics_query = """
        INSERT INTO product_characteristics (Brand_id, Brand, Descript) VALUES (%s, %s, %s);
        """
        for prod_image_name in prod_images_names:
            cursor.execute(insert_characteristics_query, prod_image_name)

        conn.commit()


        print('xxxx')
    except Exception as error:
        print(f"Error: {error}")
        conn.rollback()
    finally:
        cursor.close()
        conn.close()

In [60]:
def get_image_links(prod_soup):
    
    item_list_1 = prod_soup.select('ul li button img')

    ##### get image
    images = []
    i = 0
    for item in item_list_1: 
            

        # 3 from back, 11 is item, 10 close up
        if (i == 2) | (i == 9) | (i == 10):
            images = images + [item['src']]
        
        i += 1
    
    
    return images

In [82]:
def get_description(prod_soup):
    
    description = prod_soup.select('div.Description_description__IDgi6 p')[0].text
    description = description + ' ' + prod_soup.select('div.ProductDetail_properties__UStvB div.PropertiesContent_content__T7Hg5 p')[0].text
    

    return description

    

In [12]:
def links_to_images(links,prod_name,container_client):

    ua = UserAgent()
    header = {'User-Agent':str(ua.chrome)}

    i = 0
    for img_link in links:
        response = requests.get('https:' + img_link, headers = header)
        img = Image.open(BytesIO(response.content))

        # Resize the image to 600
        img_resized = img.resize((500, 500))

        img_name = f"Gucci{prod_name}_{i}.jpg"

        # Load to Blob
        #images_to_blob(img, img_name,container_client)
        print(img_name)
        i+=1



In [13]:
def mango(category_url,n_products):

    soup = create_soup(category_url)
    item_list = soup.select('div.ProductImage_productImage__cS5d9')

    
    #container_client = connect_blob()
    container_client = True

    conn, cursor = connect_sql()


    br = 0
    for item in item_list: 
            
        if br == n_products:
            break
        else:

            #Scrapp
            prod_soup = create_soup(item.a['href'])

            prod_images_links = get_image_links(prod_soup=prod_soup)
            prod_description = get_description(prod_soup=prod_soup)

            prod_name = 'Mango_'+str(br)  
            
            # Links to image, load to blob and return prod_images_names
            prod_images_names = links_to_images(links = prod_images_links,prod_name= br,container_client=container_client)
            
            data_to_SQL(prod_name=prod_name, prod_description=prod_description,brand='HM',prod_images_names=prod_images_names,conn=conn)
            print()

        br += 1


## Execute Scrapper

In [ ]:
# 3 from back, 11 is item, 10 close up

In [ ]:
prod_link = 'https://shop.mango.com/it/it/p/donna/giacchette/giacchette/giacca-completo-100-lino_67037113?c=43'
prod_soup = create_soup()

In [80]:
prod_soup.select('div.Description_description__IDgi6 p')[0].text

'100% lino. Modello diritto. Collo a V con rever. Maniche lunghe con polsini abbottonati. Spallini. Due tasche a falda sul lato anteriore. Chiusura con doppia abbottonatura. Spacco sul retro del bordo inferiore. Fodera interna. Disponibile Plus. Look da ufficio.'

In [81]:
prod_soup.select('div.ProductDetail_properties__UStvB div.PropertiesContent_content__T7Hg5 p')[0].text

'Composizione: 100% lino. Fodera: 100% cotone. Rinforzo manic: 100% poliestere. Pistagna: 100% poliestere.'

In [83]:
get_description(prod_soup=prod_soup)

'100% lino. Modello diritto. Collo a V con rever. Maniche lunghe con polsini abbottonati. Spallini. Due tasche a falda sul lato anteriore. Chiusura con doppia abbottonatura. Spacco sul retro del bordo inferiore. Fodera interna. Disponibile Plus. Look da ufficio. Composizione: 100% lino. Fodera: 100% cotone. Rinforzo manic: 100% poliestere. Pistagna: 100% poliestere.'